In [1]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler

In [6]:
import cufflinks as cf
cf.go_offline()

In [17]:
import quandl# set up the Quandl connection
api_key = '3qg6_574job-qipYU3s_'
quandl.ApiConfig.api_key = api_key

quandl_code = "BITFINEX/BTCUSD"# get the data from the API
bitcoin_data = quandl.get(quandl_code, start_date="2005-01-01", 
                          end_date="2020-03-18", returns="numpy")# set up the data in pandas
df = pd.DataFrame(data=bitcoin_data, columns=['Date', 'High', 'Low', 'Mid', 'Last', 'Bid', 'Ask', 'Volume'])# make the 'Date' column the index
df

,Date,High,Low,Mid,Last,Bid,Ask,Volume
0,2014-04-15,513.900000,452.000000,504.23500,505.000000,503.5000,504.97,21013.584774
1,2014-04-16,547.000000,495.000000,537.50000,538.000000,537.0000,538.00,29633.358705
2,2014-04-17,538.500000,486.100000,507.02000,508.000000,506.0400,508.00,20709.783819
3,2014-04-18,509.000000,474.250000,483.77000,482.750000,482.7500,484.79,10458.045243
4,2014-04-19,513.989900,473.830000,505.01065,507.499900,502.5313,507.49,8963.618369
...,...,...,...,...,...,...,...,...
2114,2020-03-14,5733.200000,5207.600000,5285.95000,5284.933926,5285.9000,5286.00,16968.114485
2115,2020-03-15,5987.000000,5095.000000,5371.35000,5371.300000,5371.3000,5371.40,24952.829956
2116,2020-03-16,5475.091196,4467.900924,4938.40000,4938.400000,4938.3000,4938.50,41766.810285
2117,2020-03-17,5572.800000,4956.100000,5468.25000,5467.600000,5467.6000,5468.90,19870.990239


In [18]:
df.set_index('Date')['Last'].iplot()

In [19]:
df = df[['Date','Last']]
df

,Date,Last
0,2014-04-15,505.000000
1,2014-04-16,538.000000
2,2014-04-17,508.000000
3,2014-04-18,482.750000
4,2014-04-19,507.499900
...,...,...
2114,2020-03-14,5284.933926
2115,2020-03-15,5371.300000
2116,2020-03-16,4938.400000
2117,2020-03-17,5467.600000


In [20]:
df = df[['Last']]

In [21]:
mm = MinMaxScaler()
df = pd.DataFrame(mm.fit_transform(df), columns=df.columns)
df

,Last
0,0.017182
1,0.018916
2,0.017339
3,0.016013
4,0.017313
...,...
2114,0.268334
2115,0.272872
2116,0.250126
2117,0.277932


In [22]:
def LSTM_create_data(df,tgt,train_size=.85,ventanas=1):
    index_num = list(df.columns).index(tgt)
    len_var = len(df.columns)
    dataX, dataY = [], []
    df = df.values
    for i in range(len(df) - ventanas):
        a = df[i:(i + ventanas), :]#.reshape(len_var*ventanas)
        dataX.append(a)
        dataY.append(df[i + ventanas, index_num])
    X,Y = np.array(dataX), np.array(dataY)
    train_len = int(train_size*X.shape[0])
    xt,yt = X[:train_len], Y[:train_len]
    xv,yv = X[train_len:], Y[train_len:]
    print(xt.shape[0],xv.shape[0])
    #xt = xt.T.reshape((xt.shape[0],ventanas,len_var))
    #xv = xv.T.reshape((xv.shape[0],ventanas,len_var))
    return xt,yt,xv,yv 

In [23]:
xt, yt, xv, yv= LSTM_create_data(df, tgt='Last', train_size=.85,ventanas=30)

1775 314


In [43]:
xt, yt, xv, yv= LSTM_create_data(df, tgt='Adj Close', train_size=.85,ventanas=30)

4298 759


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

/home/derek/venv/Python3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/derek/venv/Python3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/derek/venv/Python3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/derek/venv/Python3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it wi

In [28]:
model = Sequential()
model.add(GRU(units=4, input_shape=(xt.shape[1],xt.shape[2]), return_sequences=True,recurrent_dropout=.3))#return le pasas las secuencia despues de ser procesada 
model.add(GRU(units=8,recurrent_dropout=.3))
model.add(Dense(1)) #neurona de salida, valor a predecir de la red 

model.summary()

xt.shape[0] * .05

model.compile(optimizer="adam",loss="mse") #algoritmo para hacer el backpropagation, compile inicialia los pesos

history = model.fit(x=xt,
    y=yt,
    batch_size=89,
    epochs=300,
    verbose=1, # pritn que esta haciendo
    callbacks=None, #opciones durante entrenamiento ver el entrenamiento, sacar checkpoins, detnerlo
    validation_split=0.0,
    validation_data=(xv,yv),#para calcular las metricas
    shuffle=True, #para revolver el dataset
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_freq=1,
    max_queue_size=10,
    workers=4, #procesadores a utilizar
    use_multiprocessing=True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [64]:
pred = pd.DataFrame(yv, columns=['real'] )

pred["estimada"] = model.predict(xv)
pred["real_Est"] = pred["real"].shift(1)

import cufflinks as cf
cf.go_offline()

pred.iplot()

In [63]:
pred

,real,estimada
0,0.276272,0.276928
1,0.278473,0.278408
2,0.277060,0.280566
3,0.285367,0.279467
4,0.285477,0.287059
...,...,...
309,0.268334,0.288609
310,0.272872,0.272516
311,0.250126,0.275043
312,0.277932,0.254576


In [49]:
pred = pd.DataFrame(yt, columns=['real'] )


pred["estimada"] = model.predict(xt)


import cufflinks as cf
cf.go_offline()

pred.iplot()

In [50]:
history.history['loss']

[0.0004878963266571485,
 0.00046404333273604724,
 0.0004234935835079277,
 0.0004514630017129862,
 0.00043410361203378143,
 0.00041448979084314383,
 0.0004528674614594423,
 0.0003836411796100307,
 0.0003800915385937502,
 0.00037076733666996825,
 0.00038525990772054493,
 0.00035688292243744207,
 0.00037276481767676927,
 0.00036401510692011353,
 0.0003297904567443766,
 0.00034143777119002345,
 0.0003569501070641834,
 0.000335121951549655,
 0.0003146351592100396,
 0.00031687112925180667,
 0.0003159030744868239,
 0.00033323728222839377,
 0.00029749002642552434,
 0.0003110855403454633,
 0.0002960911415259905,
 0.0003123519000880958,
 0.00030455447312831405,
 0.0003147341899182746,
 0.00028405185487173813,
 0.00027355925365708583,
 0.00028089607165614796,
 0.0002709278225542938,
 0.00027799030319085793,
 0.0002918891589162202,
 0.0002611795275326175,
 0.0002729442720686768,
 0.0002635540592838341,
 0.00026822654833741697,
 0.00024655878065075256,
 0.00025792570356328803,
 0.000262754495463884

In [51]:
history.history['val_loss']

[0.0005655554451336075,
 0.0005514256173824879,
 0.0005466605310111787,
 0.0005410260833787975,
 0.0005334513011658386,
 0.0005241476206418875,
 0.0005225993857487656,
 0.0005369391144960703,
 0.0005630339139176734,
 0.0005054797903615329,
 0.0005000433073546678,
 0.0005031390468572166,
 0.0005017260262285875,
 0.0004823240842655372,
 0.0004781807259698013,
 0.0004759418654291161,
 0.00047006714417284745,
 0.0004876979787435062,
 0.0004808688487162054,
 0.0004705074995868857,
 0.0004689202128218451,
 0.00048766708910572015,
 0.00047889175785150806,
 0.00044474642109096195,
 0.00045249943534701254,
 0.0004508740751178057,
 0.00044097942824751306,
 0.0004979932138875756,
 0.00046812355904661974,
 0.0004472323473387955,
 0.00045142434513312024,
 0.00044114022172029444,
 0.0004301874178749432,
 0.00042962847973789523,
 0.0004560054399615901,
 0.0004131681271710071,
 0.0004411584611093792,
 0.0004178373468224149,
 0.00041061450318903456,
 0.0004299486772072721,
 0.00044749990640620396,
 0.0

In [52]:
loss = pd.DataFrame(history.history['loss'], columns=['train_loss'])
loss['val_loss'] = history.history['val_loss']

In [53]:
loss.iplot()

In [54]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [55]:
print('Train: ',mean_absolute_error(y_true=yt,y_pred=model.predict(xt)))
print('Test: ',mean_absolute_error(y_true=yv,y_pred=model.predict(xv)))

Train:  0.006612730197523438
Test:  0.012423391030796965


In [56]:
print('Train: ',mean_absolute_error(y_true=yt,y_pred=[yt.mean()  for x in range(yt.shape[0])]))
print('Test: ',mean_absolute_error(y_true=yv,y_pred=[yv.mean()  for x in range(yv.shape[0])]))

Train:  0.14802219829252647
Test:  0.06171933818399848


In [57]:
print('Train: ',mean_squared_error(y_true=yt,y_pred=model.predict(xt)))
print('Test: ',mean_squared_error(y_true=yv,y_pred=model.predict(xv)))

Train:  0.00017988240055912016
Test:  0.0003321882584893823


In [58]:
print('Train: ',mean_squared_error(y_true=yt,y_pred=[yt.mean()  for x in range(yt.shape[0])]))
print('Test: ',mean_squared_error(y_true=yv,y_pred=[yv.mean()  for x in range(yv.shape[0])]))

Train:  0.033742440416140766
Test:  0.005807439160276287


In [59]:
print('Train: ',r2_score(y_true=yt,y_pred=model.predict(xt)))
print('Test: ',r2_score(y_true=yv,y_pred=model.predict(xv)))

Train:  0.9946689570066464
Test:  0.9427995284459288


In [60]:
def mrae(y_true,y_pred):
    index = np.where(y_true!=0)[0]
    print(y_true.shape)
    print(len(index))
    y_true = y_true[index]
    y_pred = y_pred[index]
    error = np.absolute(y_true-y_pred)/y_true
    return np.median(error)

In [61]:
print('Train: ',mrae(y_true=yt,y_pred=model.predict(xt)))
print('Test: ',mrae(y_true=yv,y_pred=model.predict(xv)))

(1775,)
1774
Train:  0.9808986264032593
(314,)
314
Test:  0.16143963864479421
